In [15]:
import sys
import re
import os
import urllib
import urllib3
import ssl
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from cStringIO import StringIO
from compiler.ast import flatten

C:\Users\Findable Turnip\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: DeprecationWarning: The compiler package is deprecated and removed in Python 3.x.


In [16]:
def cases(data):
    check = data.find('"rel" : "documents"')
    start_link = data.find('"https:', check)
    end_link = data.find('"', start_link + 1)
    link = data[start_link + 1:end_link]
    return link, end_link

In [17]:
def get_all_cases(patentnumber):
    webaddress = "https://ptabdata.uspto.gov/ptab-api/trials?patentNumber="
    url = webaddress+patentnumber
    http = urllib3.PoolManager()
    x = http.request('GET', url)
    webdata = x.data
    caseslinks = []
    while True:
        link, position = cases(webdata)
        if link:
            caseslinks.append(link)
            webdata = webdata[position:]
        else:
            break
    return caseslinks      

In [18]:
def pdfparser(data):
    fp = file(data, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    fp.close()

    return data

In [19]:
def downloadpdf(patentnumber, web):
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    
    f = urllib.urlopen(web, context=ctx)
    with open("temp.pdf", "wb") as code:
        code.write(f.read())
    
    text = pdfparser("temp.pdf")
    searchObj = re.compile( r'\b\d,\d\d\d,\d\d\d',re.IGNORECASE)
    numbers = searchObj.findall(text)
    numbers = list(set(numbers))
    patentnumber = patentnumber[:1] + ',' + patentnumber [1:4] + ',' + patentnumber[4:]
    similar_numbers = [z for z in numbers if z != patentnumber]  
    os.remove("temp.pdf")
    
    return similar_numbers

In [20]:
def numbers_lookup(patentnumber, multiplelinks):
    numbersfound = []
    for link in multiplelinks:
        http = urllib3.PoolManager()
        y = http.request('GET', link)
        casedata = y.data
        start_pdf = casedata.find('"type" : "final decision"')
        if start_pdf != -1:
            rel_download = casedata.find('download', start_pdf)
            start_pdflink = casedata.find('"https:', rel_download)
            end_pdflink = casedata.find('"', start_pdflink + 1)
            pdflink = casedata[start_pdflink + 1:end_pdflink]
            tempvalue = downloadpdf(patentnumber, pdflink)
            if tempvalue:
                numbersfound.append(tempvalue)
    
    patentnumberfound = list(set(flatten(numbersfound)))
    return patentnumberfound

In [21]:
patentnumber = '7423248'
a = get_all_cases(patentnumber)
b = numbers_lookup(patentnumber, a)
print b

['7,459,664', '5,550,677', '4,645,975']
